<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터

In [ ]:
import pandas as pd
data=pd.read_excel('삼성전자_주가_2020_2023.xlsx')

In [ ]:
data.columns

In [ ]:
df=data.loc[:200,['뉴스 식별자','제목','키워드','본문']]
df.to_excel('감성분석 라벨링{}.xlsx'.format(200))
df=data.loc[201:400,['뉴스 식별자','키워드','제목','본문']]
df.to_excel('감성분석 라벨링{}.xlsx'.format(400))
df=data.loc[401:600,['뉴스 식별자','키워드','제목','본문']]
df.to_excel('감성분석 라벨링{}.xlsx'.format(600))
df=data.loc[601:800,['뉴스 식별자','키워드','제목','본문']]
df.to_excel('감성분석 라벨링{}.xlsx'.format(800))
df=data.loc[801:1000,['뉴스 식별자','키워드','제목','본문']]
df.to_excel('감성분석 라벨링{}.xlsx'.format(1000))


In [ ]:
# KNU 한국어 감성사전
import json
import pandas as pd

with open('/content/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
  SentiWord_info = json.load(f)

sentiword_dic = pd.DataFrame(SentiWord_info)

## 키워드 벡터화